# Aplicação do Algoritmo KNN em Diagnóstico de Fertilidade

**KNN (K-Nearest Neighbors)** é um algoritmo de aprendizado de máquina usado para **classificações** e regressões.

Sua principal ideia é simples: **a decisão sobre o valor ou categoria de um item é tomada com base nos "k" itens mais próximos a ele**. Esses itens mais próximos são encontrados em um conjunto de dados que já foi classificado ou rotulado anteriormente.

**Objetivo:**

O objetivo é aplicar o algoritmo **KNN (K-Nearest Neighbors)** para prever se uma amostra de **sêmen é fértil ou infértil**.

O conjunto de dados utilizado neste estudo foi coletado de **100 amostras de sêmen**, e as características associadas a cada amostra/pacientes, como hábitos de saúde, histórico médico, condições ambientais e dados socioeconômicos, são analisadas para identificar padrões que possam indicar a fertilidade.

**Metodologia**

Este projeto será conduzido em duas fases principais, visando aplicar e comparar o desempenho do algoritmo KNN (K-Nearest Neighbors) na classificação de amostras de sêmen como fértil ou infértil. A metodologia será dividida nas seguintes etapas:

1. **Preparação e Análise Inicial dos Dados**

  O primeiro passo será carregar o conjunto de dados e realizar uma análise exploratória para entender as características presentes, como saúde, ambiente e fatores socioeconômicos, que podem influenciar a fertilidade.

  Será feita uma análise visual e estatística para identificar possíveis problemas nos dados, como valores ausentes, valores inconsistentes ou outliers.

2. **Tratamento de Viés nos Dados**

  Na segunda fase, serão feitas modificações no conjunto de dados para eliminar possíveis vieses que possam afetar o desempenho do modelo. Isso incluirá o tratamento de valores ausentes, normalização de dados, transformação de variáveis categóricas em variáveis numéricas e a remoção de outliers, se necessário.

  O objetivo é garantir que os dados estejam mais balanceados e representem de forma justa as características de fertilidade sem influências externas que possam distorcer o modelo.

3. **Aplicação do Algoritmo KNN (Com Dados Tratados)**

  Após o tratamento dos dados, será aplicado o algoritmo KNN para classificar as amostras de sêmen como fértil ou infértil. A escolha do valor de k (número de vizinhos mais próximos) será feita com base em técnicas de validação cruzada, garantindo que o modelo seja o mais eficiente possível.

  Será avaliado o desempenho do modelo utilizando métricas como acurácia, precisão, recall e F1-score.

4. **Conclusão e Recomendações**

  Com base nos resultados obtidos, será realizada uma análise final para discutir o impacto dos vieses nos dados e como a aplicação do KNN pode ser útil na classificação de fertilidade. Além disso, recomendações para futuros estudos ou melhorias no modelo serão apresentadas.

## Bibliotecas

In [87]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from scipy.stats import shapiro
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split



## Funções

In [88]:
def padronizar(x):
  return (x-np.mean(x)) /np.std(x)

In [89]:
def normalizar(x):
  return (x-np.min(x))/(np.max(x)-np.min(x))

In [90]:
from scipy.stats import shapiro
#verificar normalidade usando teste de shapiro-wilk
def verifica_normalidade (dataframe, coluna):
  coluna_data = dataframe[coluna]
  #Realizar teste de Shapiro-Wilk
  statistc, p_valor = shapiro(coluna_data)
  #Definir nivel de significancia
  nivel_significancia = 0.1
  if p_valor > nivel_significancia:
    print(f"A coluna '{coluna}' segue uma distribuição normal com p_valor {p_valor}.")
    return True
  else:
    print(f" A coluna '{coluna}' nao segue uma distribuicao normal, com p_valor '{p_valor}'. ")
    return False

## Carregamento da Dados


In [91]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [92]:

caminho_arquivo = "https://raw.githubusercontent.com/GustavoUrsino22/DATA-SCIENCE-FIAP/refs/heads/main/BASES%20DE%20DADOS/Fertility_Diagnosis.csv"

dados = pd.read_csv(caminho_arquivo)
dados2 = dados.copy()

## **Sobre o Dataset:**

In [93]:
ds_df = dados.head()
ds_df

,temporada,idade,doencas_infantis,acidente,cirurgia,febre,alcool,fumo,sentado,classe
0,-0.33,0.69,0,1,1,0,0.8,0,0.88,N
1,-0.33,0.94,1,0,1,0,0.8,1,0.31,O
2,-0.33,0.50,1,0,0,0,1.0,-1,0.50,N
3,-0.33,0.75,0,1,1,0,1.0,-1,0.38,N
4,-0.33,0.67,1,1,0,0,0.8,-1,0.50,O


**Sobre o Dataset:**

1. Temporada

  - Inverno (-1)
  - Primavera (-0.33)
  - Verão (0.33)
  - Outono (1)

2. Idade 18-36

  - Min norm (0)
  - Máx norm (1)

3. Doenças Infantis

  - Teve (0)
  - Não Teve (1)

4. Acidente ou Trauma Grave

  - Teve (0)
  - Não Teve (1)

5. Intervenção Cirurgica

  - Teve (0)
  - Não Teve (1)
  
6. Febre Alta no Último Ano

  - Menos de 3 meses (-1)
  - Mais de 3 meses (0)
  - Não Teve (1)

7. Frequência de Consumo de Alcool

  - Várias vezes ao dia (0)
  - Todos os dias (0.2)
  - Varias vezes por semana (0.40)
  - Uma vez por semana (-6)
  - Quase Nunca (0.8)
  - Nunca (1)


8. Hábito de Fumar

  - Nunca (-1)
  - Ocasionalmente (0)
  - Diáriamente (1)

9. Número de horas gastos sentado por dia (0 - 16h)

  - O horas (0)
  - 16 horas (1)

6. Output

  - Fertilidade Normal (N)
  - Fertilidade Alterada (O)

  

In [94]:
inf_df = dados.info()
inf_df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   temporada         100 non-null    float64
 1   idade             100 non-null    float64
 2   doencas_infantis  100 non-null    int64  
 3   acidente          100 non-null    int64  
 4   cirurgia          100 non-null    int64  
 5   febre             100 non-null    int64  
 6   alcool            100 non-null    float64
 7   fumo              100 non-null    int64  
 8   sentado           100 non-null    float64
 9   classe            100 non-null    object 
dtypes: float64(4), int64(5), object(1)
memory usage: 7.9+ KB


**Descrição das Variáveis**

Todas as variáveis presentes no conjunto de dados possuem 100 observações não nulas, garantindo a integridade dos dados para a análise. Além disso, a princípio, as variáveis parecem estar em seus respectivos formatos corretos, com as variáveis numéricas e categóricas adequadamente identificadas.

No entanto, será realizada uma análise mais aprofundada para verificar se há necessidade de tratamento adicional, especialmente em relação às variáveis categóricas e numéricas. Esse tratamento pode envolver a conversão de variáveis categóricas em variáveis numéricas, normalização ou padronização de variáveis numéricas, e a identificação e correção de possíveis inconsistências ou outliers.

## Tratamentos

In [95]:
dados['classe'] = dados['classe'].map({'N': 1, 'O': 0}).astype(int)
dados

,temporada,idade,doencas_infantis,acidente,cirurgia,febre,alcool,fumo,sentado,classe
0,-0.33,0.69,0,1,1,0,0.8,0,0.88,1
1,-0.33,0.94,1,0,1,0,0.8,1,0.31,0
2,-0.33,0.50,1,0,0,0,1.0,-1,0.50,1
3,-0.33,0.75,0,1,1,0,1.0,-1,0.38,1
4,-0.33,0.67,1,1,0,0,0.8,-1,0.50,0
...,...,...,...,...,...,...,...,...,...,...
95,-1.00,0.67,1,0,0,0,1.0,-1,0.50,1
96,-1.00,0.61,1,0,0,0,0.8,0,0.50,1
97,-1.00,0.67,1,1,1,0,1.0,-1,0.31,1
98,-1.00,0.64,1,0,1,0,1.0,0,0.19,1


In [96]:
dados = pd.get_dummies(dados,columns = ['temporada','doencas_infantis','acidente','cirurgia','febre', 'alcool', 'fumo'])

In [97]:
dados

,idade,sentado,classe,temporada_-1.0,temporada_-0.33,temporada_0.33,temporada_1.0,doencas_infantis_0,doencas_infantis_1,acidente_0,...,febre_0,febre_1,alcool_0.2,alcool_0.4,alcool_0.6,alcool_0.8,alcool_1.0,fumo_-1,fumo_0,fumo_1
0,0.69,0.88,1,False,True,False,False,True,False,False,...,True,False,False,False,False,True,False,False,True,False
1,0.94,0.31,0,False,True,False,False,False,True,True,...,True,False,False,False,False,True,False,False,False,True
2,0.50,0.50,1,False,True,False,False,False,True,True,...,True,False,False,False,False,False,True,True,False,False
3,0.75,0.38,1,False,True,False,False,True,False,False,...,True,False,False,False,False,False,True,True,False,False
4,0.67,0.50,0,False,True,False,False,False,True,False,...,True,False,False,False,False,True,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.67,0.50,1,True,False,False,False,False,True,True,...,True,False,False,False,False,False,True,True,False,False
96,0.61,0.50,1,True,False,False,False,False,True,True,...,True,False,False,False,False,True,False,False,True,False
97,0.67,0.31,1,True,False,False,False,False,True,False,...,True,False,False,False,False,False,True,True,False,False
98,0.64,0.19,1,True,False,False,False,False,True,True,...,True,False,False,False,False,False,True,False,True,False


## KNN

In [98]:
y = dados ['classe']
X = dados.drop(columns = ['classe'])

In [99]:
X

,idade,sentado,temporada_-1.0,temporada_-0.33,temporada_0.33,temporada_1.0,doencas_infantis_0,doencas_infantis_1,acidente_0,acidente_1,...,febre_0,febre_1,alcool_0.2,alcool_0.4,alcool_0.6,alcool_0.8,alcool_1.0,fumo_-1,fumo_0,fumo_1
0,0.69,0.88,False,True,False,False,True,False,False,True,...,True,False,False,False,False,True,False,False,True,False
1,0.94,0.31,False,True,False,False,False,True,True,False,...,True,False,False,False,False,True,False,False,False,True
2,0.50,0.50,False,True,False,False,False,True,True,False,...,True,False,False,False,False,False,True,True,False,False
3,0.75,0.38,False,True,False,False,True,False,False,True,...,True,False,False,False,False,False,True,True,False,False
4,0.67,0.50,False,True,False,False,False,True,False,True,...,True,False,False,False,False,True,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.67,0.50,True,False,False,False,False,True,True,False,...,True,False,False,False,False,False,True,True,False,False
96,0.61,0.50,True,False,False,False,False,True,True,False,...,True,False,False,False,False,True,False,False,True,False
97,0.67,0.31,True,False,False,False,False,True,False,True,...,True,False,False,False,False,False,True,True,False,False
98,0.64,0.19,True,False,False,False,False,True,True,False,...,True,False,False,False,False,False,True,False,True,False


In [111]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2)

In [112]:
y_train

,classe
65,1
1,0
18,1
48,1
36,1
...,...
43,1
22,1
72,1
15,1


In [113]:
k = 3
model = KNeighborsClassifier(n_neighbors=k)
model.fit(X_train, y_train)
print(k,model.score(X_test, y_test))

3 0.9


In [114]:
for k in range(1,20):
  model = KNeighborsClassifier(n_neighbors=k)
  model.fit(X_train, y_train)
  print(k,model.score(X_test, y_test))

1 0.8333333333333334
2 0.8333333333333334
3 0.9
4 0.8333333333333334
5 0.8666666666666667
6 0.8666666666666667
7 0.8666666666666667
8 0.8666666666666667
9 0.8666666666666667
10 0.8666666666666667
11 0.8666666666666667
12 0.8666666666666667
13 0.8666666666666667
14 0.8666666666666667
15 0.8666666666666667
16 0.8666666666666667
17 0.8666666666666667
18 0.8666666666666667
19 0.8666666666666667


In [104]:
y2 = dados2 ['classe']
X2 = dados2.drop(columns = ['classe'])

In [150]:
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.3, random_state= 41)

In [151]:
for k in range(1,20):
  model = KNeighborsClassifier(n_neighbors=k)
  model.fit(X2_train, y2_train)
  print(k,model.score(X2_test, y2_test))

1 0.8666666666666667
2 0.9333333333333333
3 0.8666666666666667
4 0.8666666666666667
5 0.8666666666666667
6 0.9333333333333333
7 0.9333333333333333
8 0.9333333333333333
9 0.9333333333333333
10 0.9333333333333333
11 0.9333333333333333
12 0.9333333333333333
13 0.9333333333333333
14 0.9333333333333333
15 0.9333333333333333
16 0.9333333333333333
17 0.9333333333333333
18 0.9333333333333333
19 0.9333333333333333
